In [1]:
#Step 1: Load the Dataset

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from torchvision.datasets import MNIST

In [17]:
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()

        # Convolution Layer 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Convolution Layer 2
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)

        # Fully Connected Layer
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)  # Output 10 classes (digits 0-9)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [18]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_data = MNIST(root='./data', train=True, download=True, transform=transform)
test_data = MNIST(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(train_data, batch_size=64, shuffle=True)
testloader = DataLoader(test_data, batch_size=64, shuffle=False)


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model
model = CNN_Model().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [20]:
import time

# Total training time
total_start_time = time.time()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for data in trainloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Only print the loss value for each epoch
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

# Calculate total training time after all epochs
total_training_time = time.time() - total_start_time
print(f"Total Training Time: {total_training_time:.2f}s")


Epoch 1, Loss: 0.1668
Epoch 2, Loss: 0.0473
Epoch 3, Loss: 0.0334
Epoch 4, Loss: 0.0234
Epoch 5, Loss: 0.0190
Epoch 6, Loss: 0.0164
Epoch 7, Loss: 0.0114
Epoch 8, Loss: 0.0105
Epoch 9, Loss: 0.0084
Epoch 10, Loss: 0.0084
Total Training Time: 174.74s


In [23]:
from sklearn.metrics import f1_score

# Evaluation on test set
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
total = 0
all_labels = []
all_preds = []

with torch.no_grad():  # Disable gradient calculation during evaluation
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Collect predictions and labels for F1 score calculation
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Calculate test metrics
test_accuracy = 100 * correct / total
test_f1 = f1_score(all_labels, all_preds, average='weighted')
test_loss = test_loss / len(testloader)

# Print each test metric on a separate line
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(f"Test F1 Score: {test_f1:.4f}")
print(f"Total Training Time: {total_training_time:.2f}s")



Test Loss: 0.0365
Test Accuracy: 99.00%
Test F1 Score: 0.9900
Total Training Time: 174.74s


In [60]:
import torch
import torchvision
from torchvision.transforms import Compose, ToTensor, Normalize
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import Adam

# Hyperparamètres
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Préparer le dataset MNIST
transform = Compose([
    ToTensor(),
    Normalize((0.5,), (0.5,))  # Normalisation pour le MNIST
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Définition du modèle RCNN pour classification
class RCNNClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super(RCNNClassifier, self).__init__()

        # Backbone convolutionnel (feature extractor)
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),  # Sortie: 32x28x28
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Sortie: 32x14x14

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # Sortie: 64x14x14
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Sortie: 64x7x7
        )

        # Region Proposal Network (simplifié pour classification uniquement)
        self.rpn = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Sortie: 128x7x7
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Sortie: 128x3x3
        )

        # Fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 3 * 3, 256),
            nn.ReLU(),
            nn.Dropout(0.5),  # Régularisation
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)  # Extraction de caractéristiques
        x = self.rpn(x)          # RPN (simplifié ici pour extraire plus de caractéristiques)
        x = self.fc_layers(x)    # Classification finale
        return x

# Initialisation du modèle
model = RCNNClassifier(num_classes=10).to(DEVICE)

# Optimiseur et fonction de perte
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

# Fonction d'entraînement
def train_model(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            # Zéro du gradient
            optimizer.zero_grad()

            # Passe avant
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Rétropropagation
            loss.backward()
            optimizer.step()

            # Suivi de la perte
            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader):.4f}")

# Fonction d'évaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")

# Entraîner et évaluer
train_model(model, train_loader, criterion, optimizer, EPOCHS)
evaluate_model(model, test_loader)


Epoch [1/10], Loss: 0.2020
Epoch [2/10], Loss: 0.0615
Epoch [3/10], Loss: 0.0429
Epoch [4/10], Loss: 0.0361
Epoch [5/10], Loss: 0.0292
Epoch [6/10], Loss: 0.0257
Epoch [7/10], Loss: 0.0221
Epoch [8/10], Loss: 0.0184
Epoch [9/10], Loss: 0.0173
Epoch [10/10], Loss: 0.0150
Accuracy: 99.14%


In [61]:
import torch
import torchvision
from torchvision.transforms import Compose, ToTensor, Normalize
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import Adam
import time  # Importing the time module to measure training time

# Hyperparamètres
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Préparer le dataset MNIST
transform = Compose([
    ToTensor(),
    Normalize((0.5,), (0.5,))  # Normalisation pour le MNIST
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Définition du modèle RCNN pour classification
class RCNNClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super(RCNNClassifier, self).__init__()

        # Backbone convolutionnel (feature extractor)
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),  # Sortie: 32x28x28
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Sortie: 32x14x14

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # Sortie: 64x14x14
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Sortie: 64x7x7
        )

        # Region Proposal Network (simplifié pour classification uniquement)
        self.rpn = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Sortie: 128x7x7
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Sortie: 128x3x3
        )

        # Fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 3 * 3, 256),
            nn.ReLU(),
            nn.Dropout(0.5),  # Régularisation
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)  # Extraction de caractéristiques
        x = self.rpn(x)          # RPN (simplifié ici pour extraire plus de caractéristiques)
        x = self.fc_layers(x)    # Classification finale
        return x

# Initialisation du modèle
model = RCNNClassifier(num_classes=10).to(DEVICE)

# Optimiseur et fonction de perte
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

# Fonction d'entraînement avec calcul du temps
def train_model(model, train_loader, criterion, optimizer, epochs):
    model.train()
    start_time = time.time()  # Enregistrer le début du temps d'entraînement
    for epoch in range(epochs):
        total_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            # Zéro du gradient
            optimizer.zero_grad()

            # Passe avant
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Rétropropagation
            loss.backward()
            optimizer.step()

            # Suivi de la perte
            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader):.4f}")

    end_time = time.time()  # Enregistrer le temps à la fin de l'entraînement
    training_time = end_time - start_time  # Calculer le temps total d'entraînement
    print(f"Total training time: {training_time:.2f} seconds")

# Fonction d'évaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")

# Entraîner et évaluer
train_model(model, train_loader, criterion, optimizer, EPOCHS)
evaluate_model(model, test_loader)


Epoch [1/10], Loss: 0.2092
Epoch [2/10], Loss: 0.0609
Epoch [3/10], Loss: 0.0464
Epoch [4/10], Loss: 0.0352
Epoch [5/10], Loss: 0.0279
Epoch [6/10], Loss: 0.0264
Epoch [7/10], Loss: 0.0207
Epoch [8/10], Loss: 0.0199
Epoch [9/10], Loss: 0.0172
Epoch [10/10], Loss: 0.0139
Total training time: 173.45 seconds
Accuracy: 99.27%
